In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
#############################
# Install necessary library #
#############################
!pip install -q datasets transformers seqeval
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12

In [ ]:
############################
# Import necessary library #
############################
import json
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from google.colab import drive

import transformers
from transformers import AutoTokenizer, TrainingArguments, Trainer, optimization, TFAutoModelForTokenClassification
from datasets import load_dataset

import matplotlib.pyplot as plt

os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
###################
# Mount the drive #
###################
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocess dataset (pii-masking-300k)


In [ ]:
###########################################
# Load another dataset (pii-masking-200k) #
###########################################
pii300kdata = load_dataset("ai4privacy/pii-masking-300k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [ ]:
pii300kdata

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [ ]:
######################
# Filter out English #
######################
def filter_language(row, lang='English'):
    return row["language"] == 'English'
pii300kdata["train"] = pii300kdata["train"].filter(filter_language)
pii300kdata["validation"] = pii300kdata["validation"].filter(filter_language)

Filter:   0%|          | 0/177677 [00:00<?, ? examples/s]

Filter:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [ ]:
pii300kdata

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 29908
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 7946
    })
})

In [ ]:
import random
from IPython.display import display, HTML
# modified from: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb#scrollTo=i3j8APAoIrI3
def show_random_elements_in_pii300kdata(set_name="train", num=1, drop=True):
    picks = []
    N = pii300kdata[set_name].num_rows
    for _ in range(num):
        pick = random.randint(0, N - 1)
        while pick in picks:
            pick = random.randint(0, N - 1)
        picks.append(pick)
    df = pd.DataFrame(pii300kdata[set_name][picks])
    if drop:
        df = df.drop('privacy_mask', axis=1)
        df = df.drop('span_labels', axis=1)
        df = df.drop('mbert_text_tokens', axis=1)
        df = df.drop('set', axis=1)
    display(HTML(df.to_html()))

show_random_elements_in_pii300kdata("train", 3, True)

,source_text,target_text,mbert_bio_labels,id,language
0,nts to track your progress and identify areas for improvement.\n- Receive constructive feedback and guidance to maximize your potential and impact.\n\n**6. Long-Term Career Planning:**\n- Collaborate with career advisors to map out a long-term career trajectory aligned with your ambitions.\n- Explore opportunities for advancement within the organization and beyond.\n\nWe believe that investing in your professional development will not only benefit you personally but also contribute to the ove,nts to track your progress and identify areas for improvement.\n- Receive constructive feedback and guidance to maximize your potential and impact.\n\n**6. Long-Term Career Planning:**\n- Collaborate with career advisors to map out a long-term career trajectory aligned with your ambitions.\n- Explore opportunities for advancement within the organization and beyond.\n\nWe believe that investing in your professional development will not only benefit you personally but also contribute to the ove,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]",47817D,English
1,": 164-80-0490\n - Telephone: +202.028.689.6915\n - Address: 106, Section 492, East Dam Crest, Pena Blanca, NM, 87041\n - Time of Appointment: 1:32pm\n\n2. [Mikji]:\n - Date of Birth: 1962-03-31T00:00:00\n - Username: mikbagh\n - Email: mikbagh@hotmail.com\n - Social Security Number: 6284829938\n - Telephone: +43 688-522.5426\n - Address: 851, Terrace 683, West Lane, Princes Ri",": [SOCIALNUMBER]\n - Telephone: [TEL]\n - Address: [BUILDING], [SECADDRESS], [STREET], [CITY], [STATE], [POSTCODE]\n - Time of Appointment: [TIME]\n\n2. [[GIVENNAME1]]:\n - Date of Birth: [BOD]\n - Username: [USERNAME]\n - Email: [EMAIL]\n - Social Security Number: [SOCIALNUMBER]\n - Telephone: [TEL]\n - Address: [BUILDING], [SECADDRESS], [STREET], Princes Ri","[O, B-SOCIALNUMBER, I-SOCIALNUMBER, I-SOCIALNUMBER, I-SOCIALNUMBER, I-SOCIALNUMBER, I-SOCIALNUMBER, O, O, O, O, B-TEL, I-TEL, I-TEL, I-TEL, I-TEL, I-TEL, I-TEL, I-TEL, I-TEL, I-TEL, O, O, O, B-BUILDING, I-SECADDRESS, I-SECADDRESS, I-SECADDRESS, I-SECADDRESS, I-SECADDRESS, I-STREET, I-STREET, I-STREET, I-STREET, I-STREET, I-STREET, I-STREET, B-CITY, I-CITY, I-CITY, I-CITY, I-CITY, B-STATE, I-POSTCODE, I-POSTCODE, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TIME, I-TIME, I-TIME, O, B-GIVENNAME1, I-GIVENNAME1, I-GIVENNAME1, O, O, O, O, O, O, O, O, O, O, B-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, I-BOD, O, O, O, O, O, O, B-USERNAME, I-USERNAME, I-USERNAME, I-USERNAME, O, B-EMAIL, ...]",52224B,English
2,"{\n ""clients"": [\n {\n ""id"": ""965759885923"",\n ""title"": ""Mr"",\n ""name"": ""Arakel"",\n ""surname"": ""Mavrotas"",\n ""username"": ""stszjmyv76"",\n ""email"": ""arakel.mavrotas@aol.com"",\n ""address"": {\n ""building"": ""964"",\n ""street"": ""Rolleri Bypass Road"",\n ""city"": ""Angels Camp"",\n ""state"": ""CA"",\n ""postcode"": ""95222"",\n ""secondary_address"": ""Ranch 29""\n },\n ""country"": ""United States"",\n ""time"": ""3""\n },\n {\n ""id"": ""6296418762"",","{\n ""clients"": [\n {\n ""id"": ""[IDCARD]"",\n ""title"": ""[TITLE]"",\n ""name"": ""[GIVENNAME1]"",\n ""surname"": ""[LASTNAME1]"",\n ""username"": ""[USERNAME]"",\n ""email"": ""[EMAIL]"",\n ""address"": {\n ""building"": ""[BUILDING]"",\n ""street"": ""[STREET]"",\n ""city"": ""[CITY]"",\n ""state"": ""[STATE]"",\n ""postcode"": ""[POSTCODE]"",\n ""secondary_address"": ""[SECADDRESS]""\n },\n ""country"": ""[COUNTRY]"",\n ""time"": ""[TIME]""\n },\n {\n ""id"": ""[IDCARD]"",","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD, O, O, O, O, O, O, O, B-TITLE

In [ ]:
all_labels = [label for entry in pii300kdata['train']['mbert_bio_labels'] for label in entry]
unique_labels = set(all_labels)
print('number of labels:', len(unique_labels))

number of labels: 57


In [ ]:
####################
# Config for model #
####################
class CFG_pii_masking_200k:
    seed = 5214                                     # seed for reproducibility (e.g., for train validate split)
    batch_size = 16                                 # batch size for model
    labels = unique_labels                          # all labels
    num_labels = len(labels)                        # number of labels
    id2labels = dict(enumerate(labels))             # id to labels
    labels2id = {v:k for k,v in id2labels.items()}  # labels to id

In [ ]:
###################
# Tokenize inputs #
###################
# modified from: https://huggingface.co/docs/transformers/en/tasks/token_classification
model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/COMP5214/project/model"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
# modified from: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb#scrollTo=vc0BSBLIIrJQ
label_all_tokens = False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["mbert_text_tokens"], truncation=True, max_length=512, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["mbert_bio_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(CFG_pii_masking_200k.labels2id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(CFG_pii_masking_200k.labels2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

example = pii300kdata["train"][0:1]
tokenize_and_align_labels(example)

{'input_ids': [[101, 4942, 1001, 1001, 15333, 6593, 1024, 2177, 6752, 1001, 1001, 12520, 2005, 4748, 1001, 1001, 3260, 1001, 1001, 1055, 2832, 2204, 2851, 1010, 3071, 1010, 1045, 3246, 2023, 4471, 4858, 2017, 2092, 1012, 2004, 2057, 3613, 2256, 9634, 1001, 1001, 1055, 6194, 1010, 1045, 2052, 2066, 2000, 10651, 2017, 2006, 1996, 6745, 8973, 1998, 3145, 2592, 1012, 3531, 2424, 2917, 1996, 2051, 1001, 1001, 2240, 2005, 2256, 9046, 6295, 1024, 1011, 1059, 1001, 1001, 1061, 2078, 1001, 1001, 1053, 1001, 1001, 27830, 1001, 1001, 1044, 1001, 1001, 1014, 1001, 1001, 5187, 1011, 3116, 2012, 2184, 1024, 2322, 1001, 1001, 2572, 1011, 11320, 2912, 1012, 20934, 1001, 1001, 1054, 2290, 1011, 3116, 2012, 2538, 1011, 1053, 1001, 1001, 6289, 2072, 1001, 1001, 1048, 1012, 15966, 1001, 1001, 19982, 1001, 1001, 9413, 1011, 3116, 2012, 4284, 2627, 2410, 1011, 1043, 2232, 1001, 1001, 19330, 2050, 1001, 1001, 1049, 2232, 1001, 1001, 9808, 3366, 1001, 1001, 1999, 1012, 22949, 1001, 1001, 10381, 1001, 1001, 17

In [ ]:
tokenize_and_align_labels(pii300kdata["train"][:5])
train_ds = pii300kdata["train"]
valid_ds = pii300kdata["validation"]
tokenized_train_dataset = train_ds.map(tokenize_and_align_labels, batched=True)
tokenized_valid_dataset = valid_ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/29908 [00:00<?, ? examples/s]

Map:   0%|          | 0/7946 [00:00<?, ? examples/s]

In [ ]:
###################
# Load DistilBert #
###################
# modified from: https://huggingface.co/docs/transformers/en/tasks/token_classification
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=CFG_pii_masking_200k.num_labels, ignore_mismatched_sizes=True)

In [ ]:
model

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
model_name = model_checkpoint.split("/")[-1]
task = 'ner'
dataset = '300k'
batch_size = 16
args = TrainingArguments(
    f"/content/drive/MyDrive/Colab Notebooks/COMP5214/project/model/{model_name}-finetuned-{task}-{dataset}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=CFG_pii_masking_200k.batch_size,
    per_device_eval_batch_size=CFG_pii_masking_200k.batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer, max_length=512) # pad the tokenized text and label to the same size (longest)

In [ ]:
###########
# Metrics #
###########
# modified from: https://huggingface.co/docs/transformers/en/tasks/token_classification
from datasets import load_metric
metric = load_metric("seqeval")
# Usage: metric.compute(predictions=[['hi', 'wa'], ['we', 'wa']], references=[['he', 'wa'], ['we', 'wa']])
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [CFG_pii_masking_200k.id2labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [CFG_pii_masking_200k.id2labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    precision = results["overall_precision"]
    recall = results["overall_recall"]
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f5" : (1 + 5*5) * precision * recall / (5*5*precision + recall),
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-19-d1b0c652a9f4>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F5,F1,Accuracy
1,0.260400,0.563264,0.166971,0.155759,0.156163,0.161171,0.843996
2,0.233200,0.573580,0.170580,0.168326,0.168411,0.169445,0.842540
3,0.220800,0.588707,0.177648,0.170981,0.171228,0.174251,0.843673
4,0.208800,0.597980,0.185114,0.182900,0.182984,0.184000,0.845123
5,0.199100,0.621298,0.172094,0.179488,0.179192,0.175713,0.840477


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Epoch,Training Loss,Validation Loss,Precision,Recall,F5,F1,Accuracy
1,0.260400,0.563264,0.166971,0.155759,0.156163,0.161171,0.843996
2,0.233200,0.573580,0.170580,0.168326,0.168411,0.169445,0.842540
3,0.220800,0.588707,0.177648,0.170981,0.171228,0.174251,0.843673
4,0.208800,0.597980,0.185114,0.182900,0.182984,0.184000,0.845123
5,0.199100,0.621298,0.172094,0.179488,0.179192,0.175713,0.840477
6,0.188400,0.628868,0.176180,0.179143,0.179027,0.177649,0.843432
7,0.191500,0.628768,0.180499,0.181302,0.181271,0.180900,0.843528
8,0.182700,0.648988,0.176343,0.184217,0.183901,0.180194,0.842078
9,0.174400,0.653273,0.184934,0.186959,0.186880,0.185941,0.844092
10,0.171500,0.663759,0.182587,0.186505,0.186352,0.184525,0.844018


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, u

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/COMP5214/project/model')

In [ ]:
################
# Model Output #
################
sample = 0

predictions, labels, _ = trainer.predict(tokenized_train_dataset[0:1])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [CFG_pii_masking_200k.labels2id[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [CFG_pii_masking_200k.labels2id[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

print(true_predictions)
print(true_labels)

datum = tokenized_train_dataset[sample]
tokens = tokenizer.convert_ids_to_tokens(datum["input_ids"])
output = []
for i, token in enumerate(tokens):
    if (true_prediction[i] == 'O'):
        output.append(token)
    else:
        output.append(true_prediction[i])

print(output)

# Preprocess dataset (dataset provided by Kaggle)

In [ ]:
#################
# Load the data #
#################
data_path = "/content/drive/MyDrive/Colab Notebooks/COMP5214/project/data" # Modify to the correct path
train_path = os.path.join(data_path, "train.json")
test_path = os.path.join(data_path, "test.json")

with open(train_path) as json_file:
    train_data = json.load(json_file)
with open(test_path) as json_file:
    test_data = json.load(json_file)